In [1]:
#Pulls all notable stock info from this specific site. Certain factors like: 3 month % change/6 month % change missing, probably need
#to pull from another site. 

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import bs4

PATH = "C:\\Users\\hobajons\\Webdrivers\\chromedriver.exe"
driver = webdriver.Chrome(PATH)

driver.get("https://www.gurufocus.com/insider/summary")

string = "//ul[@class='el-pager']/li[text()='1']"
new_character = '2'
position = -3
new_character_int = int(new_character) 
new_string = string[:position] + str(new_character_int) + string[position+1:]

important = []
date = []
data_one = []
name_role = []
buy_sell = []
categories = []
insider_name = []
Ticker = []

while new_character_int <=20: #change this number to however many pages you want (max 555). 
    
    position = -3
    new_string = string[:position] + str(new_character_int) + string[position+1:] 

    driver.find_element_by_xpath(new_string).click

    try:
        element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, new_string))
        )
        element.click()   

    except:
        time.sleep(5)
        pass 
        
    
    html = driver.page_source 

    soup = bs4.BeautifulSoup(html, 'html.parser')


    for name in soup.select(".position-company-insider-page-style"):
        name_role.append(name.text) 

    

    #returns all buy/sell on one page

    for b_s in soup.select(".table-type-info"):
        buy_sell.append(b_s.text) 

    

    #returns all categories on page

    for category in soup.select(".table-th"):
        categories.append(category.text) 

    

    #Includes both individuals that purchased, and hedgefunds. 

    for name in soup.select(".insider-buy-link"):
        insider_name.append(name.text)


    #date of transaction 

    for dates in soup.select(".table-date-info"):
        date.append(dates.text)

    #Ticker in descending order as shown on site. 

    

    for name in soup.select(".table-stock-info"):
        Ticker.append(name.text)

    #returns horizontal information on website - nearly all, includes a lot of 
    #useful numbers

    for categories in soup.find_all(class_='right-align number-field'):
        data_one.append(categories.text) 

    # current Price of stocks

    Current_Price = data_one[0::10]

    # Shares purchased/sold

    Shares_Traded = data_one[1::10]

    # % Change in ownership

    ownership_change = data_one[2::10]

    # Price of stock upon transaction

    upon_transaction = data_one[3::10]

    # Shares held now

    held_now = data_one[5::10]

    # % change since transaction 

    change_in_price = data_one[6::10]

    # Market Cap 

    market_cap = data_one[9::10]

    
    for item in zip(date,Ticker,buy_sell,change_in_price):
        important.append(item)
        
    new_character_int +=1 

In [2]:
#convert all dates into datetime, so I can compare the dates (to find days since purchase)
date_time = []

from dateutil import parser
for item in date:
    item = parser.parse(item)
    date_time.append(item)
    

In [3]:
date_time[0] - date_time[200]
#days since. Now I do this for all dates, then I create a new column with days since transaction 
#remember to get rid of weekends. 

datetime.timedelta(days=2)

In [4]:
from datetime import date
from datetime import datetime
d = date.today()
today_date = datetime.combine(d, datetime.min.time())

In [5]:
#days since purchase (business day)
i = 0
length = len(date_time)
days_since_transaction = []

while i < length:
    days_since_transaction.append(today_date-date_time[i])
    i+=1 
    

In [6]:
days_since_transaction

[datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=1),
 datetime.timedelta(days=2),
 datetime.timedelta(days=2),
 datetime.time

In [8]:
import pandas as pd

df1 = pd.DataFrame(Ticker)
df1.columns = ['Ticker Name']

df1['Date of Transaction'] = date_time
df1['Days Since Transaction'] = days_since_transaction
df1['Buy/Sell'] = buy_sell
df1['Change in Price since Transaction'] = change_in_price

In [9]:
df1

,Ticker Name,Date of Transaction,Days Since Transaction,Buy/Sell,Change in Price since Transaction
0,HPQ,2020-12-09,1 days,Sell,0.13%
1,VAPO,2020-12-09,1 days,Sell,-4.84%
2,HPE,2020-12-09,1 days,Sell,0.75%
3,HPE,2020-12-09,1 days,Sell,0.75%
4,NARI,2020-12-09,1 days,Sell,1.62%
...,...,...,...,...,...
755,NSTG,2020-12-07,3 days,Sell,4.30%
756,NSTG,2020-12-07,3 days,Sell,8.14%
757,ICE,2020-12-07,3 days,Sell,0.03%
758,OSH,2020-12-07,3 days,Sell,17.39%
